In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

%matplotlib inline

In [2]:
def order_pics(pics):
    ordered_pics = []
    ordered_targ = []
    for i in range(41):
        num = i + 1
        for pic in pics:
            if int(pic.split('_')[1].replace('.jpg','')) == num:
                ordered_pics.append(pic)
                ordered_targ.append(num)
    return ordered_pics, ordered_targ

def load_images_from_folder(folder):
    images = []
    pics = os.listdir(folder)
    pics, target = order_pics(pics)
    for filename in pics:
        img = cv2.imread(os.path.join(folder,filename)) # read image
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # transform to grayscale
        img = cv2.resize(img, (80, 80), interpolation = cv2.INTER_AREA) # resize to 80x80
        if img is not None:
            images.append(img)
    return images, target

In [4]:
folder = 'face_orl'
X, y = load_images_from_folder(folder)
X = [i.flatten() for i in X]
X = df.drop(['ID', 'isTest', 'label'], axis=1)
y = df['label']

In [5]:
X = pd.DataFrame(X)
y = pd.DataFrame(y)
y.columns = ['onehot']
Y = y.join(pd.get_dummies(y['onehot'], drop_first=False, prefix='onehot'))
Y = Y.drop('onehot', axis=1)

In [6]:
def ADMM(X, Y, W, alpha = 0.1, rou = 0.1):
    d = np.size(W, 0)  
    c = np.size(W, 1)
    n = np.size(Y, 0)
    k = 0
    if np.all(W == 0):
        A = np.zeros((d, c))
    else:
        A = W / np.linalg.norm(W, 'fro')
    sigma, eita = np.zeros((d, c)), np.zeros((n, c))
    U, V = np.zeros((d, c)), np.zeros((n, c))
    #WW = np.ones((d, c))
    while k < 10: 
    #np.linalg.norm(W - WW, 'fro') > 1:
        #WW = W
        B = alpha * A - sigma + rou * U + np.dot(X, (eita + rou * Y - rou * V))
        XI = rou * (np.dot(X, X.T) + np.eye(d))
        W = np.dot(np.linalg.inv(XI), B)
        M = W + 1 / rou * sigma
        for i in range(d):
            th = rou * np.linalg.norm(M[i])
            if alpha < th:
                U[i] = (1 - alpha / th) * M[i]
            else:
                U[i] = np.zeros(c)
        N = Y - np.dot(X.T, W) + 1 / rou * eita
        for i in range(n):
            tth = rou * np.linalg.norm(N[i])
            if 1 < tth:
                V[i] = (1 - 1 / tth) * N[i]
            else:
                V[i] = np.zeros(c)
        sigma = sigma + rou * (W - U)
        eita = eita + rou * (Y - np.dot(X.T, W) - V)
        k += 1
       # print(W)
    return W

X = np.array([[1,1,8],[1,2,6]])
Y = np.array([[1, 0], [0, 1], [0, 1]])
W = np.array([[1e-3, 1e-3],[1e-3, 1e-3]])

In [7]:
def FS_l(X, Y, alpha = 0.1):
    d = np.size(X, 0)  
    c = np.size(Y, 1)
    W = np.zeros((d, c)) 
    WWW = np.ones((d, c))
    k = 0
    while np.linalg.norm(W - WWW, 'fro') > np.linalg.norm(W, 'fro') * 1e-5:
        WWW = W
        W = ADMM(X, Y, W, alpha)
    return W
    

In [8]:
ans = pd.DataFrame((FS_l(np.array(X.T), np.array(Y), alpha=1)).T)

In [10]:
abs_mean = ans.abs().mean()

# 定义阈值
threshold = 1e-3

# 找到绝对值平均小于阈值的列的索引
columns_to_drop = abs_mean[abs_mean < threshold].index

# 删除指定的列
X = X.drop(columns=X.columns[columns_to_drop])
X = pd.DataFrame(X)
X

,法人持股比例,企业规模国标_规模,法人变更_频率,经营范围变更_频率,最晚融资时间,最早融资时间,融资次数,政府支持,专利状态_无效,专利最早发明时间,专利最晚发明时间,专利最早最晚时间间隔,人才荣誉,雏鹰企业认定数量
0,0.000000,0.0,0.001894,0.009828,34.0,35.0,0.0,0.0,0.0,38.0,5.0,0.0,0.0,0
1,0.550000,0.0,0.000000,0.001693,34.0,35.0,0.0,0.0,2.0,5.0,1.0,4.0,0.0,2
2,0.027402,2.0,5.000000,5.000000,1.0,9.0,11.0,5.0,18.0,14.0,0.0,14.0,0.0,0
3,0.337389,3.0,0.000000,0.002997,4.0,4.0,3.0,0.0,7.0,6.0,0.0,6.0,0.0,0
4,0.000000,0.0,5.000000,5.000000,34.0,35.0,0.0,0.0,2.0,14.0,1.0,13.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4964,0.207808,3.0,5.000000,5.000000,1.0,10.0,4.0,0.0,28.0,9.0,1.0,8.0,1.0,0
4965,0.047100,2.0,5.000000,0.000000,8.0,8.0,4.0,0.0,1.0,12.0,1.0,11.0,0.0,6
4966,0.000000,3.0,0.000000,0.000000,3.0,16.0,14.0,0.0,21.0,15.0,0.0,15.0,0.0,5
4967,0.000000,2.0,0.000928,0.000932,34.0,35.0,0.0,0.0,0.0,17.0,2.0,15.0,0.0,0
